# <span style="color:white; font-family:PT Sans Narrow; font-size:1.1em"> Import Packages </span>

In [20]:
%reload_ext autoreload
%autoreload 2

import os, sys 
data_dir = os.path.dirname(os.path.realpath('.'))
sys.path.append(data_dir)


from crowdkit.datasets import load_dataset
import pandas as pd
import ipywidgets
import load_data 
import funcs 

print('data_dir:', data_dir)

%reload_ext load_data
%reload_ext funcs

data_dir: /workspace/crowd-kit


2021-12-14 03:10:42.512646: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-14 03:10:42.512684: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-14 03:10:42.512705: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ws-84853f81-86bd-4b7b-bf19-ac6ba2f0dacd): /proc/driver/nvidia/version does not exist
2021-12-14 03:10:42.513140: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# <span style="color:white; opacity:0.7; font-family:PT Sans Narrow; font-size:1.3em"> 1. Loading the Data </span>
## <span style="color:green; font-family:PT Sans Narrow; font-size:1.1em"> 1.1 Selecting the Dataset </span>

In [16]:
dataset = ipywidgets.Dropdown( options = [ ('1. kr-vs-kp'    ,'kr-vs-kp'), 
                                           ('2. mushroom'    ,'mushroom'),
                                           ('3. sick'        ,'sick'),
                                           ('4. spambase'    ,'spambase'),
                                           ('5. tic-tac-toe' ,'tic-tac-toe'),
                                           ('6. splice'      ,'splice'),
                                           ('8. waveform'    ,'waveform'),
                                           ('9. biodeg'      ,'biodeg'),
                                           ('10. horse-colic','horse-colic'),
                                           ('11. ionosphere' ,'ionosphere'),
                                           ('12. vote'       ,'vote')],      
                                value = 'vote')


# @ipywidgets.interact(WHICH_DATASET = dataset)
# def read_data(WHICH_DATASET):
#     if WHICH_DATASET in ['sick','splice','biodeg','vote','horse-colic']:
#         print('dataset does not exist')

#     else:
        
WHICH_DATASET = 'ionosphere'
data, feature_columns = load_data.aim1_3_read_download_UCI_database(WHICH_DATASET=WHICH_DATASET, mode='read')

print('train shape:',data['train'].shape)
data['train'].head(3)

directory found /workspace/crowd-kit/data_mine
train shape: (281, 35)


,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a25,a26,a27,a28,a29,a30,a31,a32,a33,true
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1


In [21]:
ARLS = {'num_labelers': 20, 
        'low_dis':      0.3, 
        'high_dis':     0.9}

predicted_labels, uncertainty, true_labels, labelers_strength = funcs.apply_technique_aim_1_3( data = data, ARLS = ARLS, num_simulations = 20,  feature_columns = feature_columns)

workers: 100%|██████████| 20/20 [00:06<00:00,  2.88it/s]


In [26]:

df

,truth,labeler_0,labeler_1,labeler_2,labeler_3,labeler_4,labeler_5,labeler_6,labeler_7,labeler_8,...,labeler_10,labeler_11,labeler_12,labeler_13,labeler_14,labeler_15,labeler_16,labeler_17,labeler_18,labeler_19
0,1,1,0,1,1,0,1,0,0,1,...,1,0,1,1,1,1,1,1,1,0
3,0,0,1,1,0,0,1,0,1,0,...,0,1,0,1,1,0,1,1,0,0
4,1,0,0,1,0,0,1,1,0,1,...,1,0,0,1,0,1,1,0,1,0
5,0,1,1,0,0,1,0,0,1,0,...,0,0,1,0,1,0,0,1,1,1
6,1,0,0,1,1,0,1,1,1,0,...,1,1,1,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,1,0,0,0,1,1,0,1,0,0,...,0,0,1,1,1,1,1,1,1,0
344,1,0,0,1,1,1,1,1,0,0,...,1,0,0,1,1,1,0,1,0,1
345,1,1,0,0,1,1,1,1,0,1,...,1,0,1,1,0,1,0,0,1,1
346,1,0,0,0,1,0,0,1,1,0,...,1,0,1,1,0,1,0,0,1,1


In [68]:
# turn the below into a function

# converting the labels from binary to integer
df = true_labels['train'].astype(int)

# dropping the true label columns 
df2 = df.drop(columns=['truth']).stack()

# stacking all the labelers labels into one column 
df3 = df2.reset_index().rename(columns={'level_0':'task', 'level_1':'performer', 0:'label'})

# reordering the columns to make it similar to crowd-kit examples
df3 = df3[['performer','task','label']]
df3 

## <span style="color:Green; font-family:PT Sans Narrow; font-size:1.1em"> 1.1 Preprocessing the data to adapt to this package </span>

,performer,task,label
0,labeler_0,0,1
1,labeler_1,0,0
2,labeler_2,0,1
3,labeler_3,0,1
4,labeler_4,0,0
...,...,...,...
5615,labeler_15,350,0
5616,labeler_16,350,0
5617,labeler_17,350,1
5618,labeler_18,350,0


In [72]:
datasets_list = ['TlkAgg-Categorical']
data_mode = 'synthetic' # 'synthetic' or 'real'

df, df_gt = load_dataset('relevance-2')
df

,performer,task,label
0,w851,t30685,1
1,w6991,t30008,0
2,w2596,t36316,0
3,w5507,t15145,1
4,w2982,t44785,1
...,...,...,...
475531,w4660,t62250,1
475532,w6630,t46626,0
475533,w4605,t93513,1
475534,w1928,t29002,0


In [74]:
df_gt.shape 

(10079,)

In [ ]:
df[df.task=='t49052'].performer.shape

(5,)